In [ ]:
import cobra
from cobra.io import read_sbml_model, load_matlab_model
import corda
import pandas as pd
import numpy as np
import os



In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [ ]:
model = read_sbml_model('/Users/jiang817/Library/CloudStorage/Box-Box/Boyu-Gut_microbiome/Neuron_GEM/Draft_reconstruction/CORDA_inputs/Recon3D.xml')

In [ ]:
os.environ['GUROBI_HOME'] = "/Users/jiang817/Desktop/Gurobi_license"
os.environ['GRB_LICENSE_FILE'] = "/Users/jiang817/Desktop/Gurobi_license/gurobi.lic"

In [ ]:
Recon3D = read_sbml_model('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Neuron_GEM/Draft_reconstruction/CORDA_inputs/Recon3D.xml')
#recon3d = load_matlab_model('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Neuron_GEM/Draft_reconstruction/CORDA_inputs/Recon3D.mat')

In [ ]:
RxnScore = pd.read_csv('/depot/pbaloni/data/Lab_members/Boyu_Jiang/Neuron_GEM/Draft_reconstruction/CORDA_inputs/Reactionscore.csv', index_col=0)

In [ ]:
RxnScore[RxnScore >= q30] = 3
RxnScore[RxnScore <  q30 ] = 2
RxnScore.fillna(0, inplace=True)

In [ ]:
confidence_score = {}
for i in range(len(RxnScore.index)):
    confidence_score[RxnScore.index[i]] = RxnScore['Score'][i]

In [ ]:
confidence_score['BIOMASS_maintenance'] = 3
confidence_score['ATPS4mi'] = 3

In [ ]:
met_list = []
biomass = Recon3D.reactions.get_by_id('BIOMASS_maintenance')
atps4 = Recon3D.reactions.get_by_id('ATPS4mi')
for i in biomass.metabolites:
    met = i.id
    #print(met)
    met_list.append(met)
print(len(met_list))
for i in atps4.metabolites:
    met = i.id
    #print(met)
    met_list.append(met)
print(len(met_list))
met_list = np.unique(met_list).tolist()

In [ ]:
ts_model = corda.CORDA(Recon3D, confidence_score, met_list)

In [ ]:
ts_model.build()

In [ ]:
ts_model.included['BIOMASS_maintenance']


In [ ]:
# q30
print(ts_model)

In [ ]:
num = 0
for i in ts_model.included:
    if ts_model.included[i] == True:
        num += 1
print(num)

In [ ]:
ts_cobra_model = ts_model.cobra_model()

In [ ]:
cobra.io.write_sbml_model(ts_cobra_model, '/depot/pbaloni/data/Lab_members/Boyu_Jiang/Neuron_GEM/Draft_reconstruction/output/CORDA_draft_reconstruction.xml')